# 학습 관련된 기술들

### 다룰 주제: 가중치 매개변수의 최적값을 탐색하는 최적화 방법, 가중치 매개변수 초깃값, 하이퍼파라미터 설정 방법 등 -> 다 중요한 것들

### SGD 복습

W <- W - $\eta{\operatorname{d}\!L\over\operatorname{d}\!W}$

여기서 $\eta$(에타)는 학습률로 0.01이나 0.001임

In [2]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr#learning rate
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -=self.lr*grads[key]

### SGD의 단점

SGD는 단순하고 구현도 쉽지만, 때에 따라 비효율적일 때가 있음.

비등방성함수에는 탐색 경로가 지그재그여서 비효율적임.

### 모멘텀

모멘텀은 '운동량'을 뜻하는 단어로 물리와 관계가 있음.

W <- W + v 여기서 v는 velocity

In [4]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

### Adagrad

신경망에서는 학습률이 중요한데 전체적인 학습률 값을 낮추는 것을 발전시킨 버전이 Adagrad